In [1]:
import pandas as pd
import numpy as np

## Prepare FiveThirtyEight Data

In [2]:
df1 = pd.read_csv("https://raw.githubusercontent.com/fivethirtyeight/data/master/comic-characters/dc-wikia-data.csv")
df1['publisher'] = "DC Comics"
df1.rename(str.lower, axis='columns', inplace=True)
df1.rename(columns={'id':'identity_type', 'name':'character_name'}, inplace=True)

In [3]:
df2 = pd.read_csv("https://raw.githubusercontent.com/fivethirtyeight/data/master/comic-characters/marvel-wikia-data.csv")
df2['publisher'] = "MARVEL Comics"
df2.rename(str.lower, axis='columns', inplace=True)
df2.rename(columns={'id':'identity_type', 'name':'character_name'}, inplace=True)

In [4]:
df_fte = pd.concat([df1, df2], ignore_index=True)

In [5]:
df_fte.head()

,page_id,character_name,urlslug,identity_type,align,eye,hair,sex,gsm,alive,appearances,first appearance,year,publisher
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,"1939, May",1939.0,DC Comics
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,"1986, October",1986.0,DC Comics
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1565.0,"1959, October",1959.0,DC Comics
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,NaN,Living Characters,1316.0,"1987, February",1987.0,DC Comics
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,1237.0,"1940, April",1940.0,DC Comics


In [6]:
# ss = df_fte.duplicated(subset=['character_name', 'first appearance','year'], keep=False)
# df_fte[ss==True].sort_values(by=['character_name'])

In [7]:
# df_fte[(df_fte.character_name == 'charlie') & (df_fte.year == 1963.0)]

#### Making join column 

In [8]:
def get_year_month(first_appearance):  # 1939, May
    datetime = pd.to_datetime(first_appearance, errors='ignore', yearfirst=True) 
    try:
        dt_str = datetime.strftime('%b-%y')  # e.g. 1939-05
        return dt_str
    except ValueError:
        return np.nan
    except AttributeError:
        if datetime == "1988, Holiday":
            datetime = "1988, December"  # I guess...
            datetime = pd.to_datetime(datetime, errors='ignore', yearfirst=True)  
            dt_str = datetime.strftime('%b-%y')  # e.g. 1939-05
            return dt_str
        if type(datetime) == str:
            datetime = pd.to_datetime(datetime, errors='ignore', yearfirst=True) 
            return datetime
        return datetime

In [9]:
df_fte['first appearance'] = df_fte.apply(lambda row: get_year_month(row['first appearance']), axis=1)

In [10]:
df_fte['join_name'] = df_fte.apply(lambda row: f"{row.character_name.split('(')[0].strip().replace(' ', '_').lower()}_{str(row['first appearance']).replace('-', '_').lower()}", axis=1)

In [11]:
df_fte.shape

(23272, 15)

In [12]:

df_fte.head()

,page_id,character_name,urlslug,identity_type,align,eye,hair,sex,gsm,alive,appearances,first appearance,year,publisher,join_name
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,May-39,1939.0,DC Comics,batman_may_39
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,Oct-86,1986.0,DC Comics,superman_oct_86
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1565.0,Oct-59,1959.0,DC Comics,green_lantern_oct_59
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,NaN,Living Characters,1316.0,Feb-87,1987.0,DC Comics,james_gordon_feb_87
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,1237.0,Apr-40,1940.0,DC Comics,richard_grayson_apr_40


In [13]:
df_fte.join_name.value_counts()

ghost_rider_may_09               3
squid_feb_89                     3
michael_van_patrick_aug_07       3
peter_parker_mar_95              3
angel_of_death_apr_93            2
pisces_aug_77                    2
dave_nan                         2
reno_aug_99                      2
frankenstein's_monster_jan_68    2
fuel_nan                         2
caretaker_oct_08                 2
jason_nan                        2
carbon_oct_79                    2
gemini_jul_77                    2
laralei_aug_77                   2
devil_sep_81                     2
tomazooma_nov_68                 2
william_shakespeare_nan          2
titannus_jan_05                  2
lawrence_zaxton_aug_63           2
kerberos_jan_06                  2
komodo_nov_95                    2
thelius_nov_75                   2
hoenir_sep_76                    2
agent_x_mar_42                   2
james_madrox_may_87              2
kerwin_kronus_oct_76             2
spymaster_jan_71                 2
clown_aug_09        

This repetition is here since the characters are e.g. changing align (going from good to bad), or have died, etc.

In [14]:
df_fte[df_fte.join_name=='superman_Oct-86']

,page_id,character_name,urlslug,identity_type,align,eye,hair,sex,gsm,alive,appearances,first appearance,year,publisher,join_name


In [15]:
df_fte.drop(['character_name'], axis=1, inplace=True)

---

## Prepare ComicVine Data

In [16]:
def extract_origin_name(x):
    origin = x.get('origin', None)
    origin_name = origin.get('name', origin) if origin else x
    return origin_name if type(origin_name) == str else np.nan

In [17]:
def extract_issue_cover_date(x):
    first_appeared_in_issue = x.get('first_appeared_in_issue', None)
    issue_id = first_appeared_in_issue.get('id', None) if first_appeared_in_issue else np.nan
    return df_is[df_is.issue_id == issue_id].cover_date.values[0] if issue_id and first_appeared_in_issue else np.nan

In [18]:
df_characters = pd.read_json("../ComicVine/characters.json")
df_cv = df_characters[['name', 'birth', 'count_of_issue_appearances','aliases', 'deck', 'description', 'id', 'image', 'origin', 'real_name', 'first_appeared_in_issue']]
df_cv.rename(columns={'id':'character_id', 'name':'character_name'}, inplace=True)

df_issues = pd.read_json("../ComicVine/issues.json")
df_is = df_issues[['id', 'cover_date']]
df_is.rename(columns={'id':'issue_id'}, inplace=True)


C:\Users\Jelena Banjac\Anaconda3\lib\site-packages\pandas\core\frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
C:\Users\Jelena Banjac\Anaconda3\lib\site-packages\pandas\core\frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [19]:
df_cv['image'] = df_cv.apply(lambda row: row.image['super_url'], axis=1)
df_cv['origin'] = df_cv.apply(lambda row: extract_origin_name(row), axis=1)
df_cv['first_appeared_in_issue'] = df_cv.apply(lambda row: extract_issue_cover_date(row), axis=1)
df_cv['first_appeared_in_issue'] = df_cv.apply(lambda row: get_year_month(row['first_appeared_in_issue']), axis=1)

C:\Users\Jelena Banjac\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Jelena Banjac\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Jelena Banjac\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [20]:
df_cv.head()

,character_name,birth,count_of_issue_appearances,aliases,deck,description,character_id,image,origin,real_name,first_appeared_in_issue
0,Lightning Lad,None,903,Garth Ranzz\nLightning Boy\nProty\nLive Wire\n...,Garth Ranzz is one of the three original membe...,"<h2>Origin</h2><figure data-align=""right"" data...",1253,https://comicvine.gamespot.com/api/image/scale...,Alien,Garth Ranzz,Apr-58
1,Dream Girl,None,385,Nura Nal\nDreamy\nDreamer\nMiss Terious\nNura ...,"Nura Nal is from the planet Naltor, where all ...","<h2>Origin</h2><figure data-align=""right"" data...",1254,https://comicvine.gamespot.com/api/image/scale...,Alien,Nura Nal,Feb-64
2,Brainiac 5,None,977,Querl Dox\nBrainiac 5.1\nBrainy\nBrainiac \nB-...,Brainiac 5 is a long-standing member of the Le...,<h2>Current Events</h2><p>Brainiac 5 is presen...,1255,https://comicvine.gamespot.com/api/image/scale...,Alien,Querl Dox,May-61
3,Invisible Kid,None,387,Lyle Norg\nBrainiac 6\nInvisible Kid,"He is a member of the Legion of Super-Heroes, ...","<p data-right-indent=""0"" data-left-indent=""0"">...",1256,https://comicvine.gamespot.com/api/image/scale...,Human,Lyle Norg,Aug-60
4,Phantom Girl,None,654,Tinya Wazzo\nTinya Wazzo-Nah\nPhantom Woman\nP...,Tinya Wazzo is a native of the extra-dimension...,<h2>Current Events</h2><p>Phantom Girl current...,1257,https://comicvine.gamespot.com/api/image/scale...,Alien,Tinya Wazzo,May-61


#### Making join column 

In [21]:
df_cv['join_name'] = df_cv.apply(lambda row: f"{row.character_name.strip().replace(' ', '_').lower()}_{str(row['first_appeared_in_issue']).replace('-', '_').lower()}", axis=1)

C:\Users\Jelena Banjac\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
df_cv.join_name.value_counts()

dupe_nan                      28
emma_jun_16                    4
sarah_jul_14                   3
joe_jul_15                     3
dave_oct_14                    3
evan_jun_14                    3
silas_jul_15                   3
sarah_apr_15                   3
claire_mar_15                  3
jack_mar_16                    3
liz_may_15                     3
john_apr_14                    3
nick_oct_18                    3
shawn_nov_15                   3
alex_apr_15                    3
tom_may_17                     3
franco_apr_14                  3
adam_may_18                    3
larry_nan                      3
mara_nan                       3
finn_dec_15                    3
tiger_nan                      3
max_nan                        3
eva_nan                        3
mike_dec_13                    3
max_nov_13                     3
medusa_nan                     3
alex_aug_14                    3
sam_mar_15                     3
kokoro_dec_14                  3
          

## Joining data

In [23]:
df = pd.merge(df_fte, df_cv, on='join_name');

In [24]:
df.shape

(7408, 25)

In [40]:
df.head(2)

,page_id,urlslug,identity_type,align,eye,hair,sex,gsm,alive,appearances,first appearance,year,publisher,join_name,character_name,birth,count_of_issue_appearances,aliases,deck,description,character_id,image,origin,real_name,first_appeared_in_issue
0,1422,\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,May-39,1939.0,DC Comics,batman_may_39,Batman,"Feb 19, 1972",16603,Bruce Wayne\r\nThe Caped Crusader\r\nThe Dark ...,"Bruce Wayne, who witnessed the murder of his m...","<h2>Current Events</h2><ul><li><a href=""https:...",1699,https://comicvine.gamespot.com/api/image/scale...,Human,Bruce Wayne,May-39
1,1448,\/wiki\/Wonder_Woman_(Diana_Prince),Public Identity,Good Characters,Blue Eyes,Black Hair,Female Characters,NaN,Living Characters,1231.0,Dec-41,1941.0,DC Comics,wonder_woman_dec_41,Wonder Woman,"Mar 21, 1881",6575,Diana Prince\r\nDiana of Themyscira\r\nPrinces...,"The princess of the Amazons, armed with powers...",<h2><strong>Origin</strong></h2><figure data-a...,2048,https://comicvine.gamespot.com/api/image/scale...,Other,Diana of Themyscira,Dec-41


In [26]:
df.columns

Index(['page_id', 'urlslug', 'identity_type', 'align', 'eye', 'hair', 'sex',
       'gsm', 'alive', 'appearances', 'first appearance', 'year', 'publisher',
       'join_name', 'character_name', 'birth', 'count_of_issue_appearances',
       'aliases', 'deck', 'description', 'character_id', 'image', 'origin',
       'real_name', 'first_appeared_in_issue'],
      dtype='object')

### Checking whether tables are joined correctly

In [27]:
df.join_name.value_counts()

link_jun_94                  2
titannus_jan_05              2
micron_aug_10                2
cole_may_04                  2
branden_mar_87               2
carbon_oct_79                2
thunder_nan                  2
kerwin_kronus_oct_76         2
lorelei_nov_83               2
thelius_nov_75               2
tomazooma_nov_68             2
spymaster_jan_71             2
kerberos_jan_06              2
harmony_aug_01               2
victoria_nan                 2
komodo_nov_95                2
michael_nan                  2
lydia_jul_89                 1
morrigon_may_13              1
blitziana_mar_94             1
job_burke_dec_97             1
piper_nov_69                 1
emmael_oct_07                1
callie_nov_01                1
propaganda_nov_06            1
zeaklar_belgewater_sep_90    1
knob_may_98                  1
jefferson_jackson_jun_82     1
ixastophanis_jul_94          1
deathunt_9000_oct_92         1
                            ..
slitherogue_mar_76           1
monica_l

Check the Superman, there is 2 of them:

In [46]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 2999

In [45]:
df_fte[df_fte.join_name.str.startswith("superman") == 1]

,page_id,urlslug,identity_type,align,eye,hair,sex,gsm,alive,appearances,first appearance,year,publisher,join_name
1,23387,\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,Oct-86,1986.0,DC Comics,superman_oct_86


In [47]:
df_cv[df_cv.join_name.str.startswith("superman") == 1]

,character_name,birth,count_of_issue_appearances,aliases,deck,description,character_id,image,origin,real_name,first_appeared_in_issue,join_name
379,Superman,"Jun 18, 1971",14663,Kal-El\r\nClark Kent\r\nClark Jerome Kent\r\nC...,Rocketed to Earth as an infant from the doomed...,"<h2>Current Events</h2><figure data-align=""rig...",1807,https://comicvine.gamespot.com/api/image/scale...,Alien,Kal-El / Clark Joseph Kent,Jun-38,superman_jun_38
5928,Superman Robot,None,31,Power Man\nAjax\nAdam Newman\nMacDuff\nNed,"Ned, the last robotic replica of Superman, he ...",<h2>History</h2><p>One of the new Robot Superm...,9576,https://comicvine.gamespot.com/api/image/scale...,Robot,"""Ned""",Oct-59,superman_robot_oct_59
29455,Superman,None,35,The Superman\r\nTangent Superman (Earth-9)\r\n...,Harvey Dent has evolved into the most powerful...,"<h2>Origin</h2><figure data-align=""right"" data...",45116,https://comicvine.gamespot.com/api/image/scale...,Other,Harvey Dent,Sep-98,superman_sep_98
35501,Superman Junior,None,28,Superman Jr.\nSuper Sons,"In some story lines, Superman had a son called...",<h2>History</h2><p>Superman Junior is <a href=...,55007,https://comicvine.gamespot.com/api/image/scale...,Human,None,Dec-65,superman_junior_dec_65
36796,Superman Predator,None,1,None,Superman Predator is one of the Metapredators....,None,56708,https://comicvine.gamespot.com/api/image/scale...,Alien,None,Aug-01,superman_predator_aug_01
92304,Superman (earth-D),None,4,None,Superman of earth-D and husband of Supergirl.,<p>The Superman of earth D died fighting the S...,115385,https://comicvine.gamespot.com/api/image/scale...,Alien,Kal-El,Feb-99,superman_(earth-d)_feb_99
122511,Superman-X,None,1,None,Kryptonite-powered clone of Superman in the 41...,"<p>Superman-X was a clone of <a href=""/superma...",146555,https://comicvine.gamespot.com/api/image/scale...,Alien,Kell-El,Jan-09,superman-x_jan_09


These are two different supermans: [First One](https://comicvine.gamespot.com/character/4005-1807/) and [Second One](https://comicvine.gamespot.com/character/4005-45116/).

Now, check Sam, there is 280 of them:

In [49]:
df[df.join_name.str.startswith("batman") == 1]

,page_id,urlslug,identity_type,align,eye,hair,sex,gsm,alive,appearances,first appearance,year,publisher,join_name,character_name,birth,count_of_issue_appearances,aliases,deck,description,character_id,image,origin,real_name,first_appeared_in_issue
0,1422,\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,May-39,1939.0,DC Comics,batman_may_39,Batman,"Feb 19, 1972",16603,Bruce Wayne\r\nThe Caped Crusader\r\nThe Dark ...,"Bruce Wayne, who witnessed the murder of his m...","<h2>Current Events</h2><ul><li><a href=""https:...",1699,https://comicvine.gamespot.com/api/image/scale...,Human,Bruce Wayne,May-39


In [38]:
df[['join_name','first_appeared_in_issue', 'first appearance']].sort_values(by=['join_name'])

,join_name,first_appeared_in_issue,first appearance
6730,107_nov_94,Nov-94,Nov-94
6731,14_nov_94,Nov-94,Nov-94
5475,2-d_jan_08,Jan-08,Jan-08
5319,88_mar_94,Mar-94,Mar-94
6732,99_nov_94,Nov-94,Nov-94
7316,a'kane_dec_95,Dec-95,Dec-95
2361,a'morr_aug_05,Aug-05,Aug-05
7278,a'sai_jan_88,Jan-88,Jan-88
7331,a'yin_mar_97,Mar-97,Mar-97
1145,a-mortal_jan_97,Jan-97,Jan-97
